In [0]:
a = []
while(1):
    a.append('1')

# Connect COLAB with Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


### Checking GPU compatibility with CUDA, as you can see COLAB GPU has CUDA 10.1 installed

In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243



### Loading dataset and darknet folder to COLAB working space by unzipping file "darknet.zip"

In [3]:
!unzip "/content/drive/My Drive/darknet.zip"

Streaming output truncated to the last 5000 lines.
  inflating: __MACOSX/darknet/data/labels/._armas (128).txt  
  inflating: darknet/data/labels/armas (1280).txt  
  inflating: __MACOSX/darknet/data/labels/._armas (1280).txt  
  inflating: darknet/data/labels/armas (578).txt  
  inflating: __MACOSX/darknet/data/labels/._armas (578).txt  
  inflating: darknet/data/labels/armas (2855).txt  
  inflating: __MACOSX/darknet/data/labels/._armas (2855).txt  
  inflating: darknet/data/labels/armas (2506).txt  
  inflating: __MACOSX/darknet/data/labels/._armas (2506).txt  
  inflating: darknet/data/labels/armas (981).txt  
  inflating: __MACOSX/darknet/data/labels/._armas (981).txt  
  inflating: darknet/data/labels/armas (2156).txt  
  inflating: __MACOSX/darknet/data/labels/._armas (2156).txt  
  inflating: darknet/data/labels/armas (2678).txt  
  inflating: __MACOSX/darknet/data/labels/._armas (2678).txt  
  inflating: darknet/data/labels/armas (1957).txt  
  inflating: __MACOSX/darknet/data

### Compile darknet directory with below script:

In [4]:
%cd /content/darknet 

/content/darknet


In [5]:
!make

mkdir -p obj
mkdir -p backup
mkdir -p results
gcc -Iinclude/ -Isrc/ -DOPENCV `pkg-config --cflags opencv`  -DGPU -I/usr/local/cuda/include/ -DCUDNN  -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -c ./src/gemm.c -o obj/gemm.o
./src/gemm.c: In function ‘time_gpu’:
./src/gemm.c:232:9: warning: ‘cudaThreadSynchronize’ is deprecated [-Wdeprecated-declarations]
         cudaThreadSynchronize();
         ^~~~~~~~~~~~~~~~~~~~~
In file included from /usr/local/cuda/include/cuda_runtime.h:96:0,
                 from include/darknet.h:11,
                 from ./src/utils.h:5,
                 from ./src/gemm.c:2:
/usr/local/cuda/include/cuda_runtime_api.h:957:57: note: declared here
 extern __CUDA_DEPRECATED __host__ cudaError_t CUDARTAPI cudaThreadSynchronize(void);
                                                         ^~~~~~~~~~~~~~~~~~~~~
gcc -Iinclude/ -Isrc/ -DOPENCV `pkg-config --cflags opencv`  -DGPU -I/usr/local/cuda/include/ -DCUDNN 

In [0]:
!chmod +x ./darknet

### Save weight during training in Google Drive

This step is important since COLAB environment will be recycle after 12 hours and all files locate in its working space will be deleted. Here we defines a symbolic link to save the weight directly into our backup folder which we created in our GDrive before.

In [0]:
!rm /content/darknet/backup -r

In [0]:
!ln -s /content/drive/'My Drive'/YOLOv3_weight/backup /content/darknet

### Install dos2unix to convert train.txt, val.txt, yolo.data, yolo.names, yolov3_custom_train.cfg to unix

In [9]:
!sudo apt install dos2unix

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  dos2unix
0 upgraded, 1 newly installed, 0 to remove and 25 not upgraded.
Need to get 351 kB of archives.
After this operation, 1,267 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 dos2unix amd64 7.3.4-3 [351 kB]
Fetched 351 kB in 2s (175 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package dos2unix.
(Reading database ... 144568 files and directories curren

In [10]:
!dos2unix ./data/train.txt
!dos2unix ./data/val.txt
!dos2unix ./data/yolo.data
!dos2unix ./data/yolo.names
!dos2unix ./cfg/yolov3_custom_train.cfg

dos2unix: converting file ./data/train.txt to Unix format...
dos2unix: converting file ./data/val.txt to Unix format...
dos2unix: converting file ./data/yolo.data to Unix format...
dos2unix: converting file ./data/yolo.names to Unix format...
dos2unix: converting file ./cfg/yolov3_custom_train.cfg to Unix format...


# Finally, let's train our model

In [0]:
%cd /content/darknet
!./darknet detector train data/yolo.data cfg/yolov3_custom_train.cfg darknet53.conv.74

Streaming output truncated to the last 5000 lines.
Region 106 Avg IOU: -nan, Class: -nan, Obj: -nan, No Obj: 0.000014, .5R: -nan, .75R: -nan,  count: 0
Region 82 Avg IOU: 0.573311, Class: 0.993696, Obj: 0.099913, No Obj: 0.002563, .5R: 1.000000, .75R: 0.000000,  count: 2
Region 94 Avg IOU: -nan, Class: -nan, Obj: -nan, No Obj: 0.000036, .5R: -nan, .75R: -nan,  count: 0
Region 106 Avg IOU: -nan, Class: -nan, Obj: -nan, No Obj: 0.000012, .5R: -nan, .75R: -nan,  count: 0
Region 82 Avg IOU: 0.251510, Class: 0.994941, Obj: 0.910662, No Obj: 0.002088, .5R: 0.000000, .75R: 0.000000,  count: 1
Region 94 Avg IOU: -nan, Class: -nan, Obj: -nan, No Obj: 0.000217, .5R: -nan, .75R: -nan,  count: 0
Region 106 Avg IOU: 0.307408, Class: 0.722418, Obj: 0.000569, No Obj: 0.000017, .5R: 0.166667, .75R: 0.000000,  count: 6
Region 82 Avg IOU: 0.696515, Class: 0.994129, Obj: 0.080758, No Obj: 0.001102, .5R: 1.000000, .75R: 0.000000,  count: 1
Region 94 Avg IOU: 0.364942, Class: 0.930839, Obj: 0.403012, No Ob

### Retrain yolov3 model with saved weight

In case the COLAB server is recycled and not to train the model from beginning, we need to execute all the above steps again but this time, change darknet53.conv.74 to directory of our saved weight. For example, if we already have a weight files trained for 900 epochs, we can resume the training using the script below:

In [0]:
#!./darknet detector train data/yolo.data cfg/yolov3_custom_train.cfg backup/yolov3_900.weights